In this notebook, we will work through a Bayes Net analysis using the GES algorithm with the TETRAD software (http://www.phil.cmu.edu/tetrad/).  We will use the same dataset used for the PPI and DCM examples.

In [41]:
import os,sys
import numpy
%matplotlib inline
import matplotlib.pyplot as plt
sys.path.insert(0,'../')
from utils.mkdesign import create_design_singlecondition
from nipy.modalities.fmri.hemodynamic_models import spm_hrf,compute_regressor
from utils.make_data import make_continuous_data
from statsmodels.tsa.arima_process import arma_generate_sample
import scipy.stats
from dcm_sim import sim_dcm_dataset

results_dir = os.path.abspath("../results")
if not os.path.exists(results_dir):
    os.mkdir(results_dir)


Load the data generated using the DCM forward model. In this model, there should be a significant PPI between roi 0 and rois 2 and 4 (see the B matrix in the DCM notebook)

In [46]:
_,data_conv,params=sim_dcm_dataset()

A_mtx=params['A']
B_mtx=params['B']
u=params['u']

# downsample to 1 second TR
data=data_conv[range(0,data_conv.shape[0],int(1./params['stepsize']))]
u=[range(0,data_conv.shape[0],int(1./params['stepsize']))]
ntp=data.shape[0]

    


[[-1.  0.  0.  0.  0.]
 [ 0. -1.  0.  0.  0.]
 [ 0.  1. -1.  0.  0.]
 [ 0.  1.  0. -1.  0.]
 [ 0.  0.  0.  0. -1.]]
[[ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.]]
[[ 1.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]]


In [49]:
tetrad_dir='/home/vagrant/data/tetrad_files'
if not os.path.exists(tetrad_dir):
    os.mkdir(tetrad_dir)
    
imagesdata=numpy.hstack((numpy.array(u).T,data))
numpy.savetxt(os.path.join(tetrad_dir,"data1.txt"),
              imagesdata,delimiter='\t',
             header='u\t0\t1\t2\t3\t4',comments='')

(30000, 5)

In [29]:
data.shape

(300, 5)